**MASK WEAR DETECTION WITH LOGISTIC REGRESSION :**

First, we have to load set of images.

To begin, we will load 2 sets of 100 images (1 dataset with worn mask and 1 dataset with unworn mask).

In [26]:
import numpy as np
from PIL import Image
import os
import cv2
import torch

def load_images_from_folder(folder):
    '''load all the images from a folder'''
    images = []
    for filename in os.listdir(folder):
        '''
        Python method listdir() returns a list containing the names of the 
        entries in the directory given by path. The list is in arbitrary order. 
        It does not include the special entries '.' and '..' even if they are 
        present in the directory.'''
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = img[:,:,::-1] # To shuffle the color channels form BGR to RGB.
            # A REGARDER COMMENT CA MARCHE PLUS EN DETAIL 
            images.append(img)
    return images

#dataset_w_m = torch.from_numpy(np.array(load_images_from_folder('Worn_Mask')))
#dataset_unw_m = torch.from_numpy(np.array(load_images_from_folder('Unworn_Mask')))

dataset_w_m_array = np.array(load_images_from_folder('Worn_Mask'))
dataset_unw_m_array = np.array(load_images_from_folder('Unworn_Mask'))

In [ ]:
Show the first element of the dataset

In [27]:
dataset_w_m_array[0]

array([[[  3, 135, 148],
        [  4, 136, 149],
        [  4, 134, 148],
        ...,
        [  0, 115, 148],
        [  0, 117, 150],
        [  0, 116, 149]],

       [[  0, 130, 143],
        [  2, 134, 147],
        [  5, 135, 149],
        ...,
        [  0, 116, 149],
        [  0, 117, 150],
        [  0, 116, 149]],

       [[  0, 127, 142],
        [  0, 131, 146],
        [  3, 133, 149],
        ...,
        [  0, 116, 151],
        [  0, 116, 149],
        [  0, 114, 147]],

       ...,

       [[ 19, 150, 168],
        [ 17, 148, 166],
        [ 15, 146, 164],
        ...,
        [128, 153, 158],
        [134, 157, 163],
        [135, 160, 165]],

       [[ 16, 149, 168],
        [ 16, 149, 168],
        [ 14, 147, 166],
        ...,
        [130, 158, 162],
        [132, 157, 162],
        [130, 158, 162]],

       [[ 15, 148, 167],
        [ 16, 149, 168],
        [ 15, 148, 167],
        ...,
        [132, 160, 164],
        [132, 160, 164],
        [130, 158, 162]]

In [41]:
import torchvision.transforms as transforms

dataset_unw_m = torch.from_numpy(dataset_unw_m_array)
dataset_w_m = torch.from_numpy(dataset_w_m_array)

Show the type :

In [42]:
type(dataset_unw_m)

torch.Tensor

Study on one element :

In [43]:
img_tensor = dataset_unw_m[0]

Show its type and its shape :

In [46]:
print(type(img_tensor))
print(img_tensor.shape)

<class 'torch.Tensor'>
torch.Size([1024, 1024, 3])


In [48]:
print(img_tensor[10:15,10:15,:])
print(torch.max(img_tensor), torch.min(img_tensor))

tensor([[[204,   0,   1],
         [204,   0,   1],
         [204,   0,   1],
         [203,   0,   1],
         [203,   0,   1]],

        [[204,   0,   1],
         [204,   0,   1],
         [205,   0,   1],
         [204,   0,   1],
         [204,   0,   1]],

        [[204,   0,   3],
         [205,   0,   3],
         [206,   0,   3],
         [207,   0,   3],
         [207,   0,   2]],

        [[204,   0,   3],
         [205,   0,   4],
         [205,   0,   4],
         [207,   0,   4],
         [208,   0,   3]],

        [[202,   1,   1],
         [203,   0,   3],
         [204,   0,   3],
         [205,   0,   3],
         [205,   0,   2]]], dtype=torch.uint8)
tensor(255, dtype=torch.uint8) tensor(0, dtype=torch.uint8)


# Labelization :

In [ ]:
def labelize_worn_mask(dataset) :
    image_label = []
    
    
dataset_unw_m = torch.from_numpy(dataset_unw_m_array)
dataset_w_m = torch.from_numpy(dataset_w_m_array)

# Training and Validation Datasets

While building real world machine learning models, it is quite common to split the dataset into 3 parts:

1. **Training set** - used to train the model i.e. compute the loss and adjust the weights of the model using gradient descent.
2. **Validation set** - used to evaluate the model while training, adjust hyperparameters (learning rate etc.) and pick the best version of the model.
3. **Test set** - used to compare different models, or different types of modeling approaches, and report the final accuracy of the model.